In [1]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import time

2024-11-15 11:11:25.539215: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 11:11:25.561116: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 11:11:25.568049: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 11:11:25.591441: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-15 11:11:29.329893: W tensorflow/comp

In [2]:
from xgboost import XGBClassifier
from xgboost import XGBRegressor

## Data

In [3]:
train = np.loadtxt('/work/users/d/d/ddinh/aaco/input_data/MelbournePedestrian/MelbournePedestrian_TRAIN.txt')
X_train = train[:, 1:]
y_train = train[:, 0]
valid = np.loadtxt('/work/users/d/d/ddinh/aaco/input_data/MelbournePedestrian/MelbournePedestrian_TEST.txt')
X_valid = valid[:, 1:]
y_valid = valid[:, 0]

y_train = np.eye(11)[y_train.astype(int)]
y_valid = np.eye(11)[y_valid.astype(int)]
y_train = y_train[:, 1:]
y_valid = y_valid[:, 1:]

In [ ]:
# load pt file 
import torch 
path = '/work/users/d/d/ddinh/aaco/results/dataset_pedestrian_rollout_first_075.pt'
data = torch.load(path)
x = data['X']
y = data['y']
mask = data['mask']
action = data['Action']

In [5]:
merge_action = []
count = 0
for i in range(action.shape[0]):
    if action[i, 0] == 1:
        merge_action.append(action[i])
        count += 1
    else:
        merge_action[count - 1] += action[i]
merge_action = np.array(merge_action)
merge_action = merge_action[:,:-1]

In [6]:
merge_action

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [7]:
model = XGBClassifier()
model.load_model('/work/users/d/d/ddinh/aaco/models/pedestrian.model')

In [8]:
pred = model.predict_proba(np.concatenate([X_valid[:merge_action.shape[0]] * merge_action, merge_action], axis=1))

In [9]:
np.sum(pred)

2262.2524

In [10]:
np.mean(np.argmax(y_valid, 1) == np.argmax(pred, 1))

0.9024190241902419

In [11]:
np.argmax(pred, 1)

array([0, 0, 0, ..., 9, 3, 9])

In [12]:
y_valid.shape

(2439, 10)

In [13]:
np.mean(np.sum(merge_action, 1))

3.606806

In [14]:
0.9048790487904879
3.503485

3.503485

In [15]:
3.1111112
0.8913489134891349


0.8913489134891349

In [23]:
np.random.permutation(np.eye(5)[3:])[:, :np.random.randint(5)]

array([[0., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [45]:
import numpy as np
import torch

def generate_ball(N, d1, d2, a):
    # Ensure that only dimensions after 'a' are considered
    Ball = np.concatenate(
        [np.sum(np.random.permutation(np.eye(d1))[:, a+1:np.random.randint(a+1, d2)], 1, keepdims=True) for _ in range(N)],
        1
    )
    # Prepend zeros for the dimensions before 'a'
    Ball = np.concatenate([np.zeros((Ball.shape[0], a)), Ball], axis=1)
    
    return Ball

class random_mask_generator():
    def __init__(self, num_samples, feature_dim, num_generated_masks):
        self.num_samples = num_samples
        self.feature_dim = feature_dim
        self.num_generated_masks = num_generated_masks
        
    def __call__(self, mask_curr, a):
        # Pass 'a' to restrict generation to dimensions after 'a'
        ball = generate_ball(self.num_generated_masks, self.feature_dim, self.feature_dim, a)
        return torch.tensor(ball[:, np.random.permutation(self.num_generated_masks)[:self.num_generated_masks]], dtype=torch.float32).T
